<a href="https://colab.research.google.com/github/curiouser001/deneme/blob/master/custom16-5x5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

In [12]:
!nvidia-smi

Wed Apr 22 11:00:22 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.64.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P8    27W / 149W |     11MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
!unzip '/content/drive/My Drive/finaldataset.zip' -d 'finaldataset'

In [0]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
from fastai.vision import *
from fastai.metrics import error_rate
import torch
import torch.nn as nn
from torch.hub import load_state_dict_from_url
import timeit

In [0]:
class VGG(nn.Module):

    def __init__(self, features, num_classes=1000, init_weights=True):
        super(VGG, self).__init__()
        self.features = features
        self.avgpool = nn.AdaptiveAvgPool2d((7, 7))
        self.classifier = nn.Sequential(
            nn.Linear(512 * 7 * 7, 4096),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(4096, num_classes),
        )
        if init_weights:
            self._initialize_weights()

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.constant_(m.bias, 0)

def make_layers(cfg, batch_norm=False,k_size=3,padding_size=1):
    layers = []
    in_channels = 3
    for v in cfg:
        if v == 'M':
            layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
        else:
            conv2d = nn.Conv2d(in_channels, v, kernel_size=k_size, padding=padding_size)
            if batch_norm:
                layers += [conv2d, nn.BatchNorm2d(v), nn.ReLU(inplace=True)]
            else:
                layers += [conv2d, nn.ReLU(inplace=True)]
            in_channels = v
    return nn.Sequential(*layers)


def _vgg(arch, cfg, batch_norm, pretrained, progress,k_size,padding_size, **kwargs):
    if pretrained:
        kwargs['init_weights'] = False
    model = VGG(make_layers(cfgs[cfg], batch_norm=batch_norm,k_size=k_size,padding_size=padding_size), **kwargs)
    if pretrained:
        state_dict = load_state_dict_from_url(model_urls[arch],
                                              progress=progress)
        model.load_state_dict(state_dict)
    return model



cfgs = {
    'A': [64, 'M', 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'B': [64, 64, 'M', 128, 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'D': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512, 'M'],
    'E': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 256, 'M', 512, 512, 512, 512, 'M', 512, 512, 512, 512, 'M'],
}


In [0]:
from sklearn.metrics import roc_curve, auc




def run(data,vgg,epoch,name):
    learn = cnn_learner(data, vgg, pretrained=False, metrics=[error_rate,accuracy])
    start = timeit.default_timer()
    learn.fit_one_cycle(epoch)
    stop = timeit.default_timer()
    print(name, stop - start)
    learn.save(name)
    learn.recorder.plot_losses()
    # predict the validation set with our model
    interp = ClassificationInterpretation.from_learner(learn)
    interp.plot_confusion_matrix(title='Confusion matrix')
    interp.confusion_matrix()
    preds,y, loss = learn.get_preds(with_loss=True)

    # get accuracy
    acc = accuracy(preds, y)
    print('The accuracy is {0} %.'.format(acc))
        

    preds,y, loss = learn.get_preds(with_loss=True)

    # probs from log preds
    probs = np.exp(preds[:,1])
    # Compute ROC curve
    fpr, tpr, thresholds = roc_curve(y, probs, pos_label=1)

    # Compute ROC area
    roc_auc = auc(fpr, tpr)
    print('ROC area is {0}'.format(roc_auc))
    plt.figure()
    plt.plot(fpr, tpr, color='darkorange', label='ROC curve (area = %0.2f)' % roc_auc)
    plt.plot([0, 1], [0, 1], color='navy', linestyle='--')
    plt.xlim([-0.01, 1.0])
    plt.ylim([0.0, 1.01])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic')
    plt.legend(loc="lower right")
    return learn

In [0]:
imagesize = 224
batchsize = 64
path = 'finaldataset/finaldataset/'

tfms = get_transforms(do_flip=False,
                      flip_vert=False,
                      max_rotate=15.0,
                      max_zoom=1,
                      max_lighting=None,
                      max_warp=None,
                      p_affine=0.0,
                      p_lighting=0.0)

data = (ImageList.from_folder(path)
        .split_by_folder(train='train', valid='val')
        .label_from_folder()
        .transform(tfms, size=imagesize)
        .databunch(bs=batchsize)
        .normalize(imagenet_stats))


In [0]:
def vgg19_bn3x3(pretrained=False, progress=True,k_size=3,padding_size=1, **kwargs):
    return _vgg('vgg19_bn', 'E', True, pretrained, progress,k_size,padding_size, **kwargs)
learn_vgg19_3x3 = run(data, vgg19_bn3x3, 30,'vgg19-3x3')

In [0]:
def vgg13_bn5x5(pretrained=False, progress=True,k_size=5,padding_size=2, **kwargs):
    return _vgg('vgg13_bn', 'B', True, pretrained, progress,k_size,padding_size, **kwargs)
learn_vgg13_5x5 = run(data, vgg13_bn5x5, 30,'vgg13-5x5')

In [0]:
def vgg16_bn5x5(pretrained=False, progress=True,k_size=5,padding_size=2, **kwargs):
    return _vgg('vgg16_bn', 'A', True, pretrained, progress,k_size,padding_size, **kwargs)
learn_vgg16_5x5 = run(data, vgg16_bn5x5, 100,'vgg16-5x5')